<ins>Logical Workflow PA Project</ins>

0. Call Data from SQL Server and save to local .pkl. Filter by TBM 1 and time span ('2022/02/01', '2022/04/01')
1. Join data for a complete dataset
2. Initial Filter (by Equipment ID: TBM 1 and remove Trial Recipes), key generation, cycle time generation and EDA
3. Left Merge Alarm mapping and Alarm hisroy on Created Key.
4. Alarm Discovery: Alarm to GT allocation: Calculate PAAP start and finish times if needed. Do NOT Filter alarms by Type e.g.  FF, CF and MC, assin name in#dexes. Link these alarm name indexes that are occurring for the same green tire at those times identified as PA. Create GT to Alarm Index Mapping.
5. One Hot encode Alarm Indexes before we carry out feature selction to see relevant alarms.
6. Feature selection: Initial Filter using statistical univariate filter methods. 
7. Feature selection: Narrow down using ML Models. Identify which steps are posing the issue (i.e. is it sidewall, inner liner, is it transport time etc.)


In [1]:
import pandas as pd
from helpers import TBM, bcolors, Feature_Selection
TBM = TBM()
p = bcolors()
FS = Feature_Selection()

In [5]:
df_tbm = pd.read_pickle("./pkl/df_tbm_PAAP_times.pkl")
df_gt_to_alarm = pd.read_pickle("./pkl/pa_alarm_map_to_gt.pkl")

In [6]:
df_gt_to_alarm

,Id,list_of_alarms
0,11141585,[]
1,11141586,"[327, 926]"
2,11141588,[]
3,11141590,[927]
4,11141591,[926]
...,...,...
110579,12467918,[]
110580,12467919,[]
110581,12467920,[]
110582,12468000,[]


In [7]:
df_tbm

,ProducedOn,EquipmentID,ProductionDataId,UserGroupName,Name_of_Active_Operator,RecipeName,Name_of_Active_Production_Order,date/time,Barcode_on_build_tire,GT_Code_of_the_built_tire,...,VRL_VHMC_MES_CCPR_CYCLE_TIME_TDST,VRL_VHMC_MES_CCPR_CYCLE_TIME_SWST,VRL_VHMC_MES_CCPR_CYCLE_TIME_GTRU,Id,start_time,cycle_time_s,naive_addition_of_step_times,sum_subsequent_steps,PAAP_Finish_time,PAAP_Start_time
0,2022-02-01 06:03:43,TBM 1,1347468,Operator,Operator 065,GG224517HYPXY,T2206001N000,20220201 06:03:43,0015600676,GG224517HYPXY,...,5.498,0.0,2.480,11141585,2022-02-01 06:02:06,97,219.620006,85.318002,2022-02-01 06:02:17.681997776,2022-02-01 06:02:12.754997730
1,2022-02-01 06:04:28,TBM 1,1347469,Operator,Operator 065,GG224517HYPXY,T2206001N000,20220201 06:04:28,0015600677,GG224517HYPXY,...,5.667,0.0,2.480,11141586,2022-02-01 06:03:43,45,137.105000,36.230000,2022-02-01 06:03:51.770000100,2022-02-01 06:03:47.783000111
2,2022-02-01 06:05:33,TBM 1,1347470,Operator,Operator 065,GG224517HYPXY,T2206001N000,20220201 06:05:33,0015600678,GG224517HYPXY,...,5.654,0.0,2.397,11141588,2022-02-01 06:04:28,65,92.177000,43.610000,2022-02-01 06:04:49.389999986,2022-02-01 06:04:45.452999950
3,2022-02-01 06:06:24,TBM 1,1347471,Operator,Operator 065,GG224517HYPXY,T2206001N000,20220201 06:06:24,0015600679,GG224517HYPXY,...,5.572,0.0,2.448,11141590,2022-02-01 06:05:33,51,102.747001,34.745000,2022-02-01 06:05:49.255000234,2022-02-01 06:05:45.262000203
4,2022-02-01 06:07:52,TBM 1,1347472,Operator,Operator 065,GG224517HYPXY,T2206001N000,20220201 06:07:52,0015600680,GG224517HYPXY,...,5.718,0.0,2.468,11141591,2022-02-01 06:06:24,88,186.372003,79.013001,2022-02-01 06:06:32.986998677,2022-02-01 06:06:29.088998675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110579,2022-05-16 05:53:49,TBM 1,1458476,DEFAULT,DEFAULT,GG215017ULAXY,T22210001Y00,20220516 05:53:49,0016499089,GG215017ULAXY,...,5.394,0.0,1.917,12467918,2022-05-16 05:53:10,39,76.988000,30.845001,2022-05-16 05:53:18.154999495,2022-05-16 05:53:14.217999459
110580,2022-05-16 05:54:27,TBM 1,1458477,DEFAULT,DEFAULT,GG215017ULAXY,T22210001Y00,20220516 05:54:27,0016499090,GG215017ULAXY,...,5.436,0.0,1.919,12467919,2022-05-16 05:53:49,38,72.697999,30.192000,2022-05-16 05:53:56.807999969,2022-05-16 05:53:52.876000047
110581,2022-05-16 05:55:05,TBM 1,1458478,Operator,Operator 081,GG215017ULAXY,T22210001Y00,20220516 05:55:05,0016499091,GG215017ULAXY,...,5.481,0.0,1.917,12467920,2022-05-16 05:54:27,38,72.351000,30.302000,2022-05-16 05:54:34.698000312,2022-05-16 05:54:30.771000266
110582,2022-05-16 05:57:04,TBM 1,1458479,Operator,Operator 081,GG215017ULAXY,T22210001Y00,20220516 05:57:04,0016499092,GG215017ULAXY,...,5.469,0.0,1.944,12468000,2022-05-16 05:55:05,119,152.235999,110.093999,2022-05-16 05:55:13.906001449,2022-05-16 05:55:09.924001336


In [8]:
df = df_gt_to_alarm.merge(df_tbm[['VRL_VHMC_MES_CCPR_CYCLE_TIME_PAAP', 'Id']], on='Id', how='left')
df

,Id,list_of_alarms,VRL_VHMC_MES_CCPR_CYCLE_TIME_PAAP
0,11141585,[],4.927
1,11141586,"[327, 926]",3.987
2,11141588,[],3.937
3,11141590,[927],3.993
4,11141591,[926],3.898
...,...,...,...
110579,12467918,[],3.937
110580,12467919,[],3.932
110581,12467920,[],3.927
110582,12468000,[],3.982


Apply mask for no alarms. I.e. get rid of the ones without PA issues. 

In [10]:
df_filter = FS.mask_empty_lists(df)
df_filter

,Id,list_of_alarms,VRL_VHMC_MES_CCPR_CYCLE_TIME_PAAP
1,11141586,"[327, 926]",3.987
3,11141590,[927],3.993
4,11141591,[926],3.898
6,11141640,"[522, 926]",3.994
7,11141641,"[242, 522, 926]",3.893
...,...,...,...
110212,12464393,[242],3.910
110213,12464394,[242],3.911
110214,12464395,[926],3.907
110215,12464396,[926],3.913


In [11]:
ohe = FS.alarm_one_hot_encoder(df_filter)

In [12]:
ohe

,Id,list_of_alarms,VRL_VHMC_MES_CCPR_CYCLE_TIME_PAAP,6,10,17,19,33,34,37,...,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018
1,11141586,"[327, 926]",3.987,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,11141590,[927],3.993,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11141591,[926],3.898,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,11141640,"[522, 926]",3.994,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,11141641,"[242, 522, 926]",3.893,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110212,12464393,[242],3.910,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110213,12464394,[242],3.911,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110214,12464395,[926],3.907,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110215,12464396,[926],3.913,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
ohe.to_pickle("./pkl/alarm_ohe.pkl")